In [35]:
#dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
from selenium import webdriver
import requests
import pymongo
import pandas as pd
import time

# Mars news scraping
Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [36]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
#!which chromedriver
#/usr/local/bin/chromedriver

In [37]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [38]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [39]:
#using bs to write it into html
html = browser.html
soup = bs(html, 'html.parser')
#print(soup.prettify())

In [40]:
news_title = soup.find("div", class_="content_title").text
print(news_title)
news_p = soup.find("div", class_="article_teaser_body").text
print(news_p)

After a Reset, Curiosity Is Operating Normally
NASA's Mars rover Curiosity is in good health but takes a short break while engineers diagnose why it reset its computer. 


# JPL Mars Space Images - Featured Image

In [41]:
url_spaceimage = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url_spaceimage)
time.sleep(1)
html = browser.html
soup = bs(html,'html.parser')

In [42]:
featured_image_list = []
for image in soup.find_all('div',class_="img"):
    featured_image_list.append(image.find('img').get('src'))

    #feature image
feature_Image = featured_image_list[0]

In [43]:
#feature image url 
feature_Image_url = "https://www.jpl.nasa.gov/" + feature_Image

In [44]:
feature_Image_dict = {"image": feature_Image_url}

In [45]:
# print feature image url 
print("Feature Image URL:", feature_Image_url)

Feature Image URL: https://www.jpl.nasa.gov//spaceimages/images/wallpaper/PIA23047-640x350.jpg


# Mars Weather
Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet from the page.

In [46]:
url_weathertwitter = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url_weathertwitter)

html = browser.html
soup = bs(html, 'html.parser')
#print(soup.prettify)

In [47]:
tweets = soup.find("div", class_="stream").find("ol").find_all("li", class_="js-stream-item")
for tweet in tweets:
    tweet_text = tweet.find("div", class_="js-tweet-text-container").text
    if "Sol " in tweet_text:
        mars_weather = tweet_text.strip()

mars_weather

'Sol 2313 (2019-02-07), high -14C/6F, low -71C/-95F, pressure at 8.14 hPa, daylight 06:47-18:53'

# Mars Facts

In [50]:
url_facts = 'https://space-facts.com/mars/'

In [52]:
table = pd.read_html(url_facts)
table[0]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [53]:
df_mars_facts = table[0]
df_mars_facts.columns = ["Index", "Values"]
df_mars_facts.set_index(["Index"])

,Values
Index,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [54]:
mars_html_table = df_mars_facts.to_html()
mars_html_table = mars_html_table.replace("\n", "")
mars_html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Index</th>      <th>Values</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronomers<

# Mars hemispheres photo scraping

In [49]:
url_hemisphere = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url_hemisphere)
base_hemisphere_url = 'https://astrogeology.usgs.gov'

html = browser.html
soup = bs(html, 'html.parser')
    
hemisphere_image_urls = []

links = soup.find_all("div", class_="item")

for link in links:
    img_dict = {}
    title = link.find("h3").text
    next_link = link.find("div", class_="description").a["href"]
    full_next_link = base_hemisphere_url + next_link
    
    browser.visit(full_next_link)
    
    pic_html = browser.html
    pic_soup = bs(pic_html, 'html.parser')
    
    url = pic_soup.find("img", class_="wide-image")["src"]

    img_dict["title"] = title
    img_dict["img_url"] = base_hemisphere_url + url
    print(img_dict["img_url"])
    
    hemisphere_image_urls.append(img_dict)

    hemisphere_image_urls

https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg
